### Split the data into training, test and validation set

In [1]:
cd("$(homedir())/Documents/enso_project.jl")
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\lihel\Documents\enso_project.jl`


In [2]:
using CSV, DataFrames, enso_project

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   Base.PkgId(Base.UUID("e3ecd195-ca82-5397-9546-f380c1e34951"), "NonlinearSolveBaseSparseMatrixColoringsExt")
│   Base.PkgId(Base.UUID("385e4588-a1a0-5c1d-98fa-d45bf6f8ecf9"), "LinearSolveKernelAbstractionsExt")
│   Base.PkgId(Base.UUID("3bcf3b12-2128-5d18-8b3b-bcdd6f83637b"), "WeightInitializersGPUArraysExt")
│   Base.PkgId(Base.UUID("b00db79b-61e3-50fb-b26f-2d35b2d9e4ed"), "DiffEqBaseChainRulesCoreExt")
│   Base.PkgId(Base.UUID("8913a72c-1f9b-4ce2-8d82-65094dcecaec"), "NonlinearSolve")
│   Base.PkgId(Base.UUID("7edab7de-1038-5e4f-97a7-6bfc75d44324"), "NonlinearSolveQuasiNewtonForwardDiffExt")
│   Base.PkgId(Base.UUID("1dea7af3-3e70-54e6-95c3-0bf5283fa5ed"), "OrdinaryDiffEq")
│   Base.PkgId(Base.UUID("0d7ed370-da01-4f52-bd93-41d350b8b718"), "StaticArrayInterface")
│   Base.PkgId(Base.UUID("693f0f32-89f9-59b4-b981-3d79b82ef24b"), "SparseDiffToolsEnzymeExt")
│   Base.PkgId(Base.UUID("63d416d0-6995-5965-81e0-5

In [5]:
# read embedded data.
df_embed = CSV.read("data/sst_data/sst_34_anomaly_embedded.txt", DataFrame; delim=',', ignorerepeated=true, header=0)

Row,Column1,Column2,Column3,Column4,Column5
,Float64,Float64,Float64,Float64,Float64
1,0.08,0.27,2.13,-0.33,-0.96
2,-0.2,0.86,1.81,-0.44,-0.16
3,-0.14,1.24,1.22,-0.38,-0.52
4,0.02,1.73,0.68,-1.02,-0.54
5,0.49,1.68,0.68,-1.29,-0.69
6,0.65,2.21,0.45,-1.16,-1.03
7,0.27,2.13,-0.33,-0.96,-0.68
8,0.86,1.81,-0.44,-0.16,-0.54
9,1.24,1.22,-0.38,-0.52,-0.41


We create different data splits to perform training on, as we want to compare the training success given the size of the training data set. 

We keep the size of the test data set fixed to 10%. This corresponds to more than 4 years (the literature suggests that prediction of ENSO is not reliable for more than 1 year ahead).

We vary the size of the training and validation data accordingly and yield the following splits (in %):
- 20 | 70 | 10
- 40 | 50 | 10
- 50 | 40 | 10
- 60 | 30 | 10
- 70 | 20 | 10
- 80 | 10 | 10

In [9]:
percentages = [0.2, 0.4, 0.5, 0.6, 0.7, 0.8]

for p in percentages

    # create splits
    train, val, test = enso_project.train_val_test_split(df_embed, val_percent=0.9-p , test_percent=0.1)

    # store
    p = Int64(100*p)
    CSV.write("data/sst_34_data_split_$p/train_sst_34_anomaly_embedded_$p.txt", train)
    CSV.write("data/sst_34_data_split_$p/val_sst_34_anomaly_embedded_$p.txt", val)
    CSV.write("data/sst_34_data_split_$p/test_sst_34_anomaly_embedded_$p.txt", test)
end